In [76]:
import pandas as pd

In [77]:
calls = pd.read_excel('../data/Winchester_EMS_calls_RAW.xls')
calls.head()

,IncidentNumber,CallDescription,Longitude,Latitude,CallDateTime
0,16-048239,Sick Person,-78.164122,39.171316,2016-12-01 05:35:23
1,16-048253,Falls,-78.170729,39.167610,2016-12-01 10:03:51
2,16-048320,Falls,-78.159111,39.175520,2016-12-01 16:58:41
3,16-048340,Sick Person,-78.186034,39.138891,2016-12-01 20:42:58
4,16-048354,Sick Person,-78.165961,39.187564,2016-12-01 23:16:32


In [78]:
# Dropping the IncidentNumber column as it is not needed for training
calls = calls.drop(columns=['IncidentNumber'])
# Changing CallDateTime to datetime
calls['CallDateTime'] = pd.to_datetime(calls['CallDateTime'])
calls.head()

,CallDescription,Longitude,Latitude,CallDateTime
0,Sick Person,-78.164122,39.171316,2016-12-01 05:35:23
1,Falls,-78.170729,39.167610,2016-12-01 10:03:51
2,Falls,-78.159111,39.175520,2016-12-01 16:58:41
3,Sick Person,-78.186034,39.138891,2016-12-01 20:42:58
4,Sick Person,-78.165961,39.187564,2016-12-01 23:16:32


In [79]:
# Find the unique values in the CallDescription column
calls['CallDescription'].unique()

array(['Sick Person', 'Falls', 'Unknown Problem/Person Down',
       'Overdose/Poisoning/Ingestion', 'Chest Pain (Non-Traumatic)',
       'Hemorrhage/Laceration', 'Traumatic Injury',
       'Unconscious/Fainting/Near-Fainting', 'Cardiac Arrest/Death',
       'Z-Other', 'Back Pain (Non-Traumatic)',
       'Traffic/Transportation Incident', 'Heart Problems/AICD',
       'Breathing Problem', 'Pregnancy/Childbirth/Miscarriage',
       'Abdominal Pain/Problems', 'Convulsions/Seizure',
       'Diabetic Problem', 'Stroke/CVA', 'Burns/Explosion',
       'Allergic Reaction/Stings',
       'Carbon Monoxide/Hazmat/Inhalation/CBRN',
       'No Other Appropriate Choice',
       'Industrial Accident/Inaccessible Incident/Other Entrapments (Non-Vehicle)',
       'Assault', 'Headache',
       'Psychiatric Problem/Abnormal Behavior/Suicide Attempt',
       'Stab/Gunshot Wound/Penetrating Trauma', 'Animal Bite',
       'Transfer/Interfacility/Palliative Care', 'Medical Alarm',
       'Choking', 'Eye Pro

In [80]:
# Get the values counts
calls['CallDescription'].value_counts()

CallDescription
Sick Person                                                                  8556
Breathing Problem                                                            4536
Falls                                                                        4367
Chest Pain (Non-Traumatic)                                                   3397
Unconscious/Fainting/Near-Fainting                                           2809
No Other Appropriate Choice                                                  2385
Z-Other                                                                      1906
Convulsions/Seizure                                                          1699
Hemorrhage/Laceration                                                        1543
Traffic/Transportation Incident                                              1478
Abdominal Pain/Problems                                                      1251
Overdose/Poisoning/Ingestion                                                  942


In [81]:
# Combine some of the call descriptions into more general categories
def combine_call_descriptions(call_type, mapping):
    for cat, type in mapping.items():
        if call_type in type:
            return cat
    return "Other"

In [82]:
# Mappings for specific call descriptions
mapping_specific = {
        "General Medical Issue": ["Sick Person", "Unknown Problem/Person Down", "Medical Alarm", "No Other Appropriate Choice"],
        "Cardiac & Circulatory Issues": ["Chest Pain (Non-Traumatic)", "Cardiac Arrest/Death", "Heart Problems/AICD", "Stroke/CVA"],
        "Respiratory Issues": ["Breathing Problem", "Choking", "Carbon Monoxide/Hazmat/Inhalation/CBRN"],
        "Neurological Issues": ["Convulsions/Seizure", "Headache"],
        "Diabetic & Metabolic Issues": ["Diabetic Problem", "Heat/Cold Exposure"],
        "Pregnancy & Birth-Related": ["Pregnancy/Childbirth/Miscarriage"],
        "Falls & Minor Trauma": ["Falls", "Back Pain (Non-Traumatic)", "Traumatic Injury"],
        "Serious Trauma & Wounds": ["Stab/Gunshot Wound/Penetrating Trauma", "Hemorrhage/Laceration", "Burns/Explosion"],
        "Vehicle & Industrial Accidents": ["Traffic/Transportation Incident", "Industrial Accident/Inaccessible Incident/Other Entrapments (Non-Vehicle)", "Electrocution/Lightning"],
        "Assault & Violence": ["Assault", "Animal Bite"],
        "Drowning & Environmental Exposure": ["Drowning/Diving/SCUBA Accident"],
        "Mental Health & Suicide": ["Psychiatric Problem/Abnormal Behavior/Suicide Attempt"],
        "Substance-Related Issues": ["Overdose/Poisoning/Ingestion", "Allergic Reaction/Stings"],
        "Non-Urgent Transport & Checks": ["Transfer/Interfacility/Palliative Care", "Well Person Check", "Standby"]
    }

mapping_general = {
        "Medical Emergencies": ["Sick Person", "Unknown Problem/Person Down", "Medical Alarm", "No Other Appropriate Choice", 
                "Chest Pain (Non-Traumatic)", "Cardiac Arrest/Death", "Heart Problems/AICD", "Stroke/CVA",
                "Breathing Problem", "Choking", "Carbon Monoxide/Hazmat/Inhalation/CBRN",
                "Convulsions/Seizure", "Headache", "Diabetic Problem", "Heat/Cold Exposure",
                "Pregnancy/Childbirth/Miscarriage"],
        "Trauma & Injury": ["Falls", "Back Pain (Non-Traumatic)", "Traumatic Injury", "Stab/Gunshot Wound/Penetrating Trauma", 
                "Hemorrhage/Laceration", "Burns/Explosion", "Traffic/Transportation Incident", 
                "Industrial Accident/Inaccessible Incident/Other Entrapments (Non-Vehicle)", "Electrocution/Lightning", 
                "Assault", "Animal Bite", "Drowning/Diving/SCUBA Accident"],
        "Behavioral & Psychiatric Issues": ["Psychiatric Problem/Abnormal Behavior/Suicide Attempt"],
        "Non-Urgent Calls": ["Overdose/Poisoning/Ingestion", "Allergic Reaction/Stings", "Transfer/Interfacility/Palliative Care", 
                "Well Person Check", "Standby"]
}

In [83]:
# Apply the function to the CallDescription column  
calls_specific_col = calls['CallDescription'].apply(combine_call_descriptions, mapping=mapping_specific)
calls_general_col = calls['CallDescription'].apply(combine_call_descriptions, mapping=mapping_general)

In [84]:
# Create two new dataframes with each mapping
calls_specific = calls.copy()
calls_general = calls.copy()

calls_specific['CallDescription'] = calls_specific_col
calls_general['CallDescription'] = calls_general_col

In [85]:
# Save the new dataframes to csv files
calls_specific.to_csv('../data/clean/calls_specific.csv', index=False)
calls_general.to_csv('../data/clean/calls_general.csv', index=False)
calls.to_csv('../data/clean/calls_full.csv', index=False)